In [4]:
# Reprodutibilidade e checagem de versões
import os, random, numpy as np
import tensorflow as tf

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


In [6]:
# Fonte pública 284k linhas
URL = "https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv"
df = pd.read_csv(URL)
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [7]:
print("Dimensões:", df.shape)  # (n amostras, n colunas)
print("\nColunas:", list(df.columns))
print("\nAlvo:", "Class (0 = não fraude, 1 = fraude)")

# Número de features
n_features = df.shape[1] - 1
print("\nNº de features:", n_features)

# Checar nulos
print("\nNulos por coluna (deve ser tudo 0):")
print(df.isna().sum())

# Distribuição das classes
class_counts = df['Class'].value_counts()
class_pct = df['Class'].value_counts(normalize=True)*100
print("\nDistribuição da classe:")
print(pd.DataFrame({"count": class_counts, "%": class_pct.round(4)}))

# Estatísticas rápidas
df.describe().T.head(10)


Dimensões: (284807, 31)

Colunas: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']

Alvo: Class (0 = não fraude, 1 = fraude)

Nº de features: 30

Nulos por coluna (deve ser tudo 0):
Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

Distribuição da classe:
        count        %
Class                 
0      284315  99.8273
1         492   0.1727


,count,mean,std,min,25%,50%,75%,max
Time,284807.0,9.481386e+04,47488.145955,0.000000,54201.500000,84692.000000,139320.500000,172792.000000
V1,284807.0,1.168375e-15,1.958696,-56.407510,-0.920373,0.018109,1.315642,2.454930
V2,284807.0,3.416908e-16,1.651309,-72.715728,-0.598550,0.065486,0.803724,22.057729
V3,284807.0,-1.379537e-15,1.516255,-48.325589,-0.890365,0.179846,1.027196,9.382558
V4,284807.0,2.074095e-15,1.415869,-5.683171,-0.848640,-0.019847,0.743341,16.875344
V5,284807.0,9.604066e-16,1.380247,-113.743307,-0.691597,-0.054336,0.611926,34.801666
V6,284807.0,1.487313e-15,1.332271,-26.160506,-0.768296,-0.274187,0.398565,73.301626
V7,284807.0,-5.556467e-16,1.237094,-43.557242,-0.554076,0.040103,0.570436,120.589494
V8,284807.0,1.213481e-16,1.194353,-73.216718,-0.208630,0.022358,0.327346,20.007208
V9,284807.0,-2.406331e-15,1.098632,-13.434066,-0.643098,-0.051429,0.597139,15.594995


In [8]:
# Separar X e y
X = df.drop(columns=['Class'])
y = df['Class'].astype(np.int32)

# Split estratificado para manter proporção de fraudes no teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=SEED, stratify=y
)

# Padronização
scaler = StandardScaler(with_mean=True, with_std=True)
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

X_train.shape, X_test.shape


((227845, 30), (56962, 30))

In [9]:
import tensorflow as tf

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.tp = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.fp = self.add_weight(name='fp', initializer='zeros', dtype=tf.float32)
        self.fn = self.add_weight(name='fn', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred >= self.threshold, tf.float32)

        tp = tf.reduce_sum(y_true * y_pred)
        fp = tf.reduce_sum((1.0 - y_true) * y_pred)
        fn = tf.reduce_sum(y_true * (1.0 - y_pred))

        self.tp.assign_add(tp)
        self.fp.assign_add(fp)
        self.fn.assign_add(fn)

    def result(self):
        precision = self.tp / (self.tp + self.fp + 1e-7)
        recall    = self.tp / (self.tp + self.fn + 1e-7)
        f1 = 2.0 * precision * recall / (precision + recall + 1e-7)
        return f1

    def reset_states(self):
        for v in self.variables:
            v.assign(0.0)


In [10]:
from tensorflow import keras
from tensorflow.keras import layers

input_dim = X_train.shape[1]

model = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', F1Score(name='f1', threshold=0.5)]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31 (124.00 B)

 Trainable params: 31 (124.00 B)

 Non-trainable params: 0 (0.00 B)

## Explicação dos componentes do treinamento

* **Modelo `Sequential`**
  Estrutura linear de camadas: define o fluxo do *forward* e do *backprop* durante o aprendizado.

* **Camada `Dense(1)`**
  Combinação linear das features com 1 unidade → equivalente a uma regressão logística para classificar 0/1.

* **Ativação `sigmoid`**
  Converte o score em **probabilidade** entre 0 e 1 (interpretação direta como “chance de ser classe 1”).

* **Otimizador `adam`**
  Descida do gradiente com momentos adaptativos (média/variância do gradiente). Tende a treinar rápido e de forma estável.

* **Função de perda `binary_crossentropy`**
  Erro entre probabilidade prevista e rótulo binário. É o **sinal** que guia as atualizações dos pesos.

* **Métrica `accuracy`**
  Percentual de acertos. Intuitiva, porém pode ser enganosa em **dados desbalanceados**.

* **Métrica `F1`**
  Média harmônica de **Precisão** e **Recall**. Mais informativa quando a classe positiva é rara (ex.: fraude).

* **Épocas (`epochs=50`)**
  Quantas passadas completas no conjunto de treino. Mais épocas podem ajudar, mas aumentam risco de *overfitting*.

* **Tamanho do lote (`batch_size=10`)**
  Nº de exemplos por atualização de gradiente. Lotes menores geram mais atualizações por época.

* **Padronização (StandardScaler)**
  Coloca as features em escala semelhante (média 0, desvio 1), facilitando a otimização.

* **Divisão estratificada (`stratify=y`)**
  Mantém a proporção das classes em treino/teste — crucial em dados desbalanceados.

* **Limiar de decisão (0,5)**
  Probabilidades ≥ 0,5 → classe 1; abaixo → classe 0. Pode ser ajustado para otimizar F1, precisão ou recall.


In [11]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=10,
    verbose=1
)


Epoch 1/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9451 - f1: 0.0781 - loss: 0.1789
Epoch 2/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - accuracy: 0.9992 - f1: 0.7272 - loss: 0.0043
Epoch 3/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9992 - f1: 0.7358 - loss: 0.0042
Epoch 4/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step - accuracy: 0.9992 - f1: 0.7353 - loss: 0.0041
Epoch 5/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9992 - f1: 0.7332 - loss: 0.0041
Epoch 6/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step - accuracy: 0.9992 - f1: 0.7370 - loss: 0.0041
Epoch 7/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9992 - f1: 0.7370 - loss: 0.0041
Epoch 8/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.9992 - f1: 0.7359 - loss: 0.0041
Epoch 9/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9992 - f1: 0.7365 - loss: 0.0041
Epoch 10/50
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - accu

In [12]:
# Probabilidades e rótulos previstos (threshold 0.5)
y_proba = model.predict(X_test).ravel()
y_pred  = (y_proba >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
f1  = f1_score(y_test, y_pred, zero_division=0)

print(f"Accuracy (teste): {acc:.6f}")
print(f"F1 (teste):       {f1:.6f}\n")

print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred, digits=4))


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 885us/step
Accuracy (teste): 0.999105
F1 (teste):       0.735751

Matriz de confusão:
[[56840    24]
 [   27    71]]

Relatório de classificação:
              precision    recall  f1-score   support

           0     0.9995    0.9996    0.9996     56864
           1     0.7474    0.7245    0.7358        98

    accuracy                         0.9991     56962
   macro avg     0.8734    0.8620    0.8677     56962
weighted avg     0.9991    0.9991    0.9991     56962



## Interpretação dos resultados

Treinei um modelo básico (uma camada `Dense` com `sigmoid`, `adam`, `binary_crossentropy`) por **50 épocas** com **batch size = 10**.

**No teste:**
- **Acurácia:** **0,9991**
- **F1 (classe 1 – fraude):** **~0,736**
- **Matriz de confusão:**
  - Verdadeiros Negativos (TN): **56.840**
  - Falsos Positivos (FP): **24**
  - Falsos Negativos (FN): **27**
  - Verdadeiros Positivos (TP): **71**
- **Classe 1 (fraude):**
  - **Precisão:** ~0,747
  - **Recall:** ~0,725
  - **F1:** ~0,736

### O que isso quer dizer
- A **acurácia é muito alta**, mas o conjunto é **desbalanceado** (quase tudo é “não fraude”). Por isso, só olhar acurácia **pode enganar**.
- O **F1 ~0,736** é mais justo para a classe “fraude”, porque junta precisão e recall.
- O modelo é **conservador**: faz **poucos alarmes falsos** (24), mas **deixa passar algumas fraudes** (27).

### Conclusão rápida
Para um modelo linear simples, o resultado ficou **ok**: ele quase não acusa errado, porém **poderia encontrar mais fraudes**. Em casos reais, muitas vezes vale priorizar **recall** (deixar escapar menos fraude), aceitando um pouco mais de falsos positivos.

## Possíveis melhorias

1. **Ajustar o limiar (threshold):** hoje é 0,5. Baixar para **~0,30–0,45** pode **aumentar o recall** e pode melhorar o **F1**.
2. **`class_weight`:** dar mais peso à classe 1 no `model.fit` para o modelo **prestar mais atenção** às fraudes.
3. **Reamostragem no treino:** **oversampling/SMOTE** da classe 1 (ou undersampling da 0) para reduzir o desbalanceamento.
4. **Features simples:** testar **log no `Amount`** e alguma variável de **tempo** (ex: hora do dia). Mesmo num modelo linear, pode ajudar.
5. **Validação melhor:** usar **Stratified K-Fold** para ter métricas mais estáveis (principalmente o F1 da classe 1).
6. **Métricas extras:** além de accuracy/F1, relatar **ROC-AUC** e **PR-AUC** (esta é ótima quando a classe positiva é rara).